# Dimensión mensajero

In [1]:
import pandas as pd
import yaml
from sqlalchemy import create_engine
from datetime import date

### Conexión con la base de datos

In [2]:
with open('../config.yml', 'r') as f:
  config = yaml.safe_load(f)
  config_oltp = config['fuente']
  config_olap = config['bodega']

url_oltp = (f"{config_oltp['drivername']}://{config_oltp['user']}:{config_oltp['password']}@{config_oltp['host']}:"f"{config_oltp['port']}/{config_oltp['dbname']}")
url_olap = (f"{config_olap['drivername']}://{config_olap['user']}:{config_olap['password']}@{config_olap['host']}:"f"{config_olap['port']}/{config_olap['dbname']}")
#print(url_oltp)
#print(url_olap)
oltp_conn = create_engine(url_oltp)
olap_conn = create_engine(url_olap)

### Módulo de extracción

In [67]:
dim_mensajero = pd.read_sql_table('clientes_mensajeroaquitoy', oltp_conn)
table_user = pd.read_sql_table('auth_user', oltp_conn)
table_ciudad = pd.read_sql_table('ciudad', oltp_conn)
table_departamento = pd.read_sql_table('departamento', oltp_conn)
#dim_cliente = pd.read_sql_table('cliente', oltp_conn)
#dim_sede.info()
dim_mensajero.head()
#table_departamento.head()

,id,user_id,activo,fecha_entrada,fecha_salida,salario,telefono,ciudad_operacion_id,token_Firebase,url_foto
0,1,2,True,NaT,NaT,NaN,310-300000,13.0,None,http:
1,42,410,True,NaT,NaT,NaN,310-300000,1.0,None,http:
2,48,447,True,2024-07-12,NaT,1300000.0,310-300000,1.0,None,http:
3,41,400,True,NaT,NaT,NaN,310-300000,1.0,None,http:
4,13,331,True,2021-11-08,NaT,1160000.0,310-300000,4.0,None,http:


### Módulo de transformación

In [68]:
table_departamento = table_departamento.rename(columns={'nombre' : 'departamento'})
table_ciudad = table_ciudad.rename(columns={'nombre' : 'ciudad'})
table_ciudad = pd.merge(table_ciudad, table_departamento, how='inner') # reemplazo de las llaves foraneas de la tabla ciudad (elimina departamento_id)
table_ciudad = table_ciudad.drop(columns=['departamento_id']) # organizar

dim_mensajero = dim_mensajero.drop(columns=['token_Firebase', 'url_foto'])
dim_mensajero = pd.merge(dim_mensajero, table_ciudad, how='inner', left_on='ciudad_operacion_id', right_on='ciudad_id')
dim_mensajero = dim_mensajero.drop(columns=['ciudad_operacion_id', 'ciudad_id'])

table_user = table_user.drop(columns=['password', 'is_superuser', 'is_staff'])

dim_mensajero = dim_mensajero.rename(columns={'id': 'id_mensajero'})
dim_mensajero = pd.merge(dim_mensajero, table_user, how='inner', left_on='user_id', right_on='id') # ¿inner?
dim_mensajero = dim_mensajero.drop(columns=['user_id', 'id', 'activo', 'is_active'])

dim_mensajero.sort_values(by=['id_mensajero'])
#table_user.head()


,id_mensajero,fecha_entrada,fecha_salida,salario,telefono,ciudad,departamento,last_login,username,first_name,last_name,email,date_joined
0,1,NaT,NaT,NaN,310-300000,ACOPI YUMBO,VALLE DEL CAUCA,2023-12-28 22:40:25+00:00,mensajero1,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,2023-06-15 21:32:52+00:00
35,3,2012-05-08,NaT,3000000.0,310-300000,CALI,VALLE DEL CAUCA,2024-08-31 17:04:04.542037+00:00,Biil-Gates,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,2023-06-16 00:23:37+00:00
15,4,2018-12-17,NaT,1160000.0,310-300000,CALI,VALLE DEL CAUCA,2024-06-28 10:43:40.654443+00:00,Lionel_messi,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,2023-06-21 01:58:42+00:00
12,7,NaT,NaT,NaN,310-300000,CALI,VALLE DEL CAUCA,2024-08-31 13:16:35.491216+00:00,admin,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,2023-06-15 16:06:05+00:00
8,8,2021-05-18,NaT,1160000.0,310-300000,PALMIRA,VALLE DEL CAUCA,2024-08-08 20:28:39.962222+00:00,Luis Castro,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,2023-10-05 15:34:53+00:00
28,9,2015-02-16,NaT,1160000.0,310-300000,CALI,VALLE DEL CAUCA,2024-08-04 00:17:29.054331+00:00,DIEGO Maradona,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,2023-07-06 12:06:36+00:00
25,10,2019-09-02,NaT,1160000.0,310-300000,CALI,VALLE DEL CAUCA,NaT,Luis Valenciano,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,2023-10-05 15:40:15+00:00
30,11,2023-02-01,NaT,1160000.0,310-300000,CALI,VALLE DEL CAUCA,2024-08-31 14:04:08.699380+00:00,Carlos Valderrama,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,2023-10-05 15:42:11+00:00
21,12,2018-03-07,NaT,1160000.0,310-300000,CALI,VALLE DEL CAUCA,2024-08-24 02:08:44.987977+00:00,Faustino Asprilla,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,2023-10-05 15:44:20+00:00
4,13,2021-11-08,NaT,1160000.0,310-300000,PASTO,NARIÑO,2024-08-10 00:14:08.315931+00:00,GEOVANNY Hidalgo,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,2023-10-05 15:46:48+00:00


### Módulo de carga a la bodega de datos